In [37]:
# Cell 1 — Imports & Hyperparameters
import os
import time
from ultralytics import YOLO

# --------------------- Hyperparameters ---------------------
EPOCHS = 25            # You can increase to 50-100 for better accuracy
IMGSZ = 640             # Image size
BATCH = 8               # Batch size
DEVICE = 0              # GPU id, or 'cpu' for CPU
RUN_NAME = "yolov8_custom_run"

# --------------------- Paths ---------------------
BASE = r"C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking"
DATA_YAML = os.path.join(BASE, "data", "data.yaml")       # dataset.yaml location
PRETRAIN = os.path.join(BASE, "models", "yolov8n.pt")     # pretrained weights
PROJECT = os.path.join(BASE, "runs")                      # where results will be saved


In [38]:
# Cell 2 — Validate paths & Initialize model
print("🔍 Checking paths...")
print("YAML Path:", DATA_YAML, "| Exists:", os.path.exists(DATA_YAML))
print("Pretrained Weights Path:", PRETRAIN, "| Exists:", os.path.exists(PRETRAIN))
print("Project Save Path:", PROJECT)

# --- Validate dataset YAML ---
if not os.path.exists(DATA_YAML):
    raise FileNotFoundError(f"❌ dataset.yaml not found at {DATA_YAML}. Check your path or file name.")

# --- Warn if pretrained weights are missing ---
if not os.path.exists(PRETRAIN):
    print("⚠️ Warning: pretrained weights not found at", PRETRAIN, "- Ultralytics will download YOLOv8n base automatically.")

# --- Model Initialization ---
MODEL_BASE = PRETRAIN
model = YOLO(MODEL_BASE)
print("✅ YOLOv8 model loaded successfully!")


🔍 Checking paths...
YAML Path: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\data.yaml | Exists: True
Pretrained Weights Path: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\models\yolov8n.pt | Exists: True
Project Save Path: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\runs
✅ YOLOv8 model loaded successfully!


In [53]:
# Imports
import os
import time
from ultralytics import YOLO
import yaml

# ---------------------------
# Paths
# ---------------------------

BASE = r"C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking"

DATA_YAML = os.path.join(BASE, "data", "data.yaml")  # dataset YAML
PRETRAIN = os.path.join(BASE, "models", "yolov8n.pt")  # base YOLOv8n weights
PROJECT = os.path.join(BASE, "runs")                   # folder to save training runs
RUN_NAME = "yolov8_finetune"                           # name of this training run

# ---------------------------
# Training settings
# ---------------------------
EPOCHS = 50        # increase for better results
IMGSZ = 640        # input image size
BATCH = 8          # reduce if GPU memory is limited
DEVICE = 0         # 0 for GPU, 'cpu' for CPU

# ---------------------------
# Load pretrained YOLOv8 model
# ---------------------------
model = YOLO(PRETRAIN)
print("✅ Model loaded:", PRETRAIN)

# ---------------------------
# Validate train and val directories
# ---------------------------
with open(DATA_YAML, 'r') as f:
    data_yaml_content = yaml.safe_load(f)

train_images_path = r"C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\images\training"
val_images_path   = r"C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\images\val"

# Create directories if they do not exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)

# Check if train and val directories are not empty
if not os.listdir(train_images_path):
    raise FileNotFoundError(f"Training images directory exists but is empty: {train_images_path}\nPlease add training images and labels in YOLO format.")
if not os.listdir(val_images_path):
    raise FileNotFoundError(f"Validation images directory exists but is empty: {val_images_path}\nPlease add validation images and labels in YOLO format.")

print("✅ Train and Val directories exist and are not empty, ready to start training!")

# ---------------------------
# Start training
# ---------------------------
start_time = time.time()
model.train(
    data=DATA_YAML,      # dataset YAML
    epochs=EPOCHS,       # number of epochs
    imgsz=IMGSZ,         # image size
    batch=BATCH,         # batch size
    device=DEVICE,       # GPU or CPU
    project=PROJECT,     # folder to save runs
    name=RUN_NAME,       # folder name for this run
    pretrained=True      # fine-tune from base model
)

print(f"✅ Training complete! Time elapsed: {time.time() - start_time:.2f} seconds")


✅ Model loaded: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\models\yolov8n.pt
✅ Train and Val directories exist and are not empty, ready to start training!
New https://pypi.org/project/ultralytics/8.3.217 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.208  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce MX570, 2048MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, i

In [54]:
# Cell 4 — Model Evaluation & Confusion Matrix + Accuracy + Confidence
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

# Run validation on val set
results = model.val(
    data=DATA_YAML,
    batch=BATCH,
    device=DEVICE,
    imgsz=IMGSZ
)

print("✅ Validation complete!")

# --- Summary metrics ---
results.summary()  # prints precision, recall, mAP50, mAP50-95 per class

# --- Per-class metrics ---
# Extract per-class metrics from the Metric object
metrics_dict = {
    'Precision': results.box.p,
    'Recall': results.box.r,
    'F1': results.box.f1,
    'AP50': results.box.ap50,  # AP at IoU=0.5
    'AP': results.box.ap       # AP at IoU=0.5:0.95
}
metrics_df = pd.DataFrame(metrics_dict)
metrics_df.index = [results.names[i] for i in range(len(metrics_df))]
metrics_df.rename_axis("Class", inplace=True)
display(metrics_df)

# --- Build Confusion Matrix ---
print("⚠️ Confusion matrix and confidence distribution are not available directly from YOLOv8 Metric object.")
print("You can use the per-class metrics table above for evaluation.")


Ultralytics 8.3.208  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce MX570, 2048MiB)


Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.40.1 ms, read: 327.4184.0 MB/s, size: 2588.2 KB)
val: Scanning C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\labels\val.cache... 17 images, 0 backgrounds, 9 corrupt: 100% ━━━━━━━━━━━━ 17/17 8.6Kit/s 0.0s
val: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\images\val\pexels-rdne-stock-project-4921398.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 3. Possible class labels are 0-2
val: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\images\val\pexels-sairam-rasa-3419331.jpg: ignoring corrupt image/label: Label class 4 exceeds dataset class count 3. Possible class labels are 0-2
val: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\images\val\pexels-shazard-r-10594822.jpg: ignoring corrupt image/label: Label class 4 excee

,Precision,Recall,F1,AP50,AP
Class,,,,,
person,1.000000,0.00,0.000000,0.516739,0.456550
bottle,0.891171,0.75,0.814514,0.748861,0.674387
cell phone,1.000000,0.00,0.000000,0.002509,0.001505


⚠️ Confusion matrix and confidence distribution are not available directly from YOLOv8 Metric object.
You can use the per-class metrics table above for evaluation.


In [2]:
# ----------------------------------------------------
# 02_YOLO_model_training.ipynb
# ----------------------------------------------------

# Imports
import os
import time
from ultralytics import YOLO
import yaml

# -----------------------------
# Paths
# -----------------------------
BASE = r"C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking"

DATA_YAML = os.path.join(BASE, "data", "data.yaml")
PRETRAIN = os.path.join(BASE, "models", "yolov8m.pt")   # use 'm' model (larger, better accuracy)
PROJECT = os.path.join(BASE, "runs")
RUN_NAME = "yolov8_finetune_v2"                         # new run name

# -----------------------------
# Training Settings
# -----------------------------
EPOCHS = 100        # increase to learn better features
IMGSZ = 640
BATCH = 8
DEVICE = 0          # GPU: 0 , CPU: 'cpu'

# -----------------------------
# Load Model
# -----------------------------
model = YOLO(PRETRAIN)
print("✅ Loaded pretrained model:", PRETRAIN)

# -----------------------------
# Show dataset info
# -----------------------------
with open(DATA_YAML, 'r') as f:
    data_cfg = yaml.safe_load(f)

print("\nDataset Configuration:")
print(data_cfg)

# -----------------------------
# Start Training
# -----------------------------
start_time = time.time()

results = model.train(
    data=DATA_YAML, 
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    project=PROJECT,
    name=RUN_NAME,
    pretrained=True,
    patience=20,               # early stopping if no improvement
    optimizer="AdamW",         # better optimizer for small data
    lr0=0.001,                 # slightly lower learning rate
    augment=True,              # enable augmentations
    dropout=0.2,               # add dropout for regularization
    mosaic=1.0,                # use full mosaic augmentation
    mixup=0.2,                 # mixup helps generalization
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # color augmentations
)

print(f"\n✅ Training complete! Time elapsed: {(time.time() - start_time)/60:.2f} minutes")
print("Results saved in:", os.path.join(PROJECT, RUN_NAME))

# -----------------------------
# Evaluate trained model
# -----------------------------
metrics = model.val(data=DATA_YAML, device=DEVICE, imgsz=IMGSZ)
print("\n📊 Validation Results:")
print(metrics)


✅ Loaded pretrained model: C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\models\yolov8m.pt

Dataset Configuration:
{'train': 'C:/Users/ksush/Downloads/Summer_intern/Real_time_object_detection_tracking/data/images/training', 'val': 'C:/Users/ksush/Downloads/Summer_intern/Real_time_object_detection_tracking/data/images/val', 'nc': 4, 'names': ['person', 'bottle', 'cell phone', 'spoon']}
New https://pypi.org/project/ultralytics/8.3.217 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.208  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce MX570, 2048MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\ksush\Downloads\Summer_intern\Real_time_object_detection_tracking\data\data.yaml, degrees=0.0, deterministic=True, 